In [1]:
import pandas as pd
import numpy as np
import datetime 
from workalendar.usa import Massachusetts
from dateutil.parser import parse
from datetime import timedelta


path1 ='/Users/Shuxian/Documents/CSYE7390-Spring2016/CY2014 COB Interval data 1'# use your path
path2 =r'/Users/Shuxian/Documents/CSYE7390-Spring2016/CY2014 COB Interval data 2'
def readCSV(path,name):
    rowfile = pd.read_csv(path + "/"+name)
    return rowfile

In [2]:
def savefile(path,name):
    if not os.path.exists(path+"/screwed/"):
        os.makedirs(path+"/screwed/")
    newdata.to_csv(path+"/screwed/"+name,index=False)

In [3]:
import os

def getfilelist(path):
    filelist=[]
    for file in os.listdir(path):
        if file.endswith(".csv"):
            filelist.append(file)
    return filelist

In [4]:
filelist1=getfilelist(path1)
filelist2=getfilelist(path2)
#filelist3 = getfilelist(path3)
files=filelist1+filelist2

In [8]:
temp= pd.read_csv('/Users/Shuxian/Documents/CSYE7390-Spring2016/COB Interval data 2014/temp.csv')
temp=temp[0:364]
temp=temp[temp.columns[1:4]]

tm=[]
for s in temp['DATE']:
    s=str(s)
    s=datetime.datetime.strptime(s, '%Y%m%d').strftime('%m/%d/%Y')
    tm.append(s)
temp['Date']=tm

temp.drop('DATE', axis=1, inplace=True)
temp=temp[['Date', 'TMAX', 'TMIN']]
temp.head()

,Date,TMAX,TMIN
0,01/01/2014,29,14
1,01/02/2014,26,3
2,01/03/2014,15,-9
3,01/04/2014,27,-16
4,01/05/2014,37,8


In [37]:
#Build date preprocessing function
def clean(path,file):
#read file
    test=readCSV(path,file)
#drop channel    
    test.drop('Channel', axis=1, inplace=True)
#fill N/A with 0 and drop duplicates
    test.fillna(0, inplace=True)
    test=test.drop_duplicates()
    
#Add Day, Month columns  
    daylist=[]
    monthlist=[]
    year=0
    for date in test['Date']:
        date=datetime.datetime.strptime(date, '%m/%d/%Y')
#        date=parse(date)
        daylist.append(date.day)
        monthlist.append(date.month)
        year=date.year
    test['Day']=daylist
    test['Month']=monthlist
#Tracking workingday and holiday
    cal = Massachusetts()
#holidays = cal.holidays(year)
    bl=[]
    week=[]
    for date in test['Date']:
        date=parse(date)
        bl.append(cal.is_holiday(date))
        week.append(cal.is_working_day(date))
    test['Holiday'] = bl
    test['Workingday']=week
#groupby columns by sum
    test=test.groupby(['Account','Date','Day','Month','Workingday','Holiday','Units']).mean()*test.groupby(['Account','Date','Day','Month','Workingday','Holiday','Units']).count()
    test.reset_index(level = ['Account','Date','Day','Month','Workingday','Holiday','Units'], inplace = True)
#Calculate hourly rates
    time1=datetime.datetime.strptime(test.columns[8], '%H:%M')
    time2=datetime.datetime.strptime(test.columns[7], '%H:%M')
    timediff=time1-time2
    num=datetime.timedelta(hours=1)/timediff

    for i in range(24):
        test[i+1]=np.mean(test[test.columns[7+num*i:(7+num)+num*i]],axis=1)
    test=test.drop(test.columns[7:(7+num)+num*23],axis=1,inplace=False)
#Transpose table    
    test = test.reset_index(drop=True)
    
    results=[]
    for date in test['Date'].unique():
        perday=test[test['Date']==date]
        a=perday[perday.columns[7:31]]
        b=a.transpose()
        b.reset_index(level =0, inplace = True)
        header=['Hour','Power Factor','kVARh','kWh']
        b.columns = header
        b=b[header]
        arest=perday[perday.columns[0:6]]
        result = pd.concat([arest, b], axis=1)
        result.fillna(method='ffill', inplace=True)
        result.fillna(method='bfill', inplace=True)
        results.append(result)
        
    results = pd.concat(results)
    results=results.reset_index(drop=True)
    
#    newdt = results.merge(temp, on='Date', how='left')
# Get Category column
    if 'COB' in file:
        pos=file.find('-')
        nm=file[pos+1:pos+5]
        if '.' in nm:
            nm = nm.replace(".", "")
            results['Category']=nm
        results['Category']=nm
    else:
        nm=file.partition(' ')[0]
        results['Category']=nm
#save    
    if not os.path.exists(path+"/screwed/"):
        os.makedirs(path+"/screwed/")
    results.to_csv(path+"/screwed/"+file,index=False)

In [38]:
for file in filelist1:
    clean(path1, file)

### Merge datasets

In [30]:
path3='/Users/Shuxian/Documents/CSYE7390-Spring2016/CY2014 COB Interval data 1/done'
path4 ='/Users/Shuxian/Documents/CSYE7390-Spring2016/CY2014 COB Interval data 2/done'
mfiles=getfilelist(path3) 

In [31]:
newdt=[]
for file in mfiles:
    newfile = pd.read_csv(path3 + "/"+file)
    newfile = newfile.reset_index(drop=True)
    newdt.append(newfile)

In [ ]:
newdt=pd.concat(newdt)

In [ ]:
newdt=newdt.reset_index(drop=True)
newdt.to_csv(path3+'/mergedata2.csv', index=False)

In [ ]:
merge1= pd.read_csv(path3 + '/mergedata1.csv')
merge2=pd.read_csv(path4 + '/mergedata2.csv')

In [ ]:
finalmerged=pd.concat([merge1,merge2])
finalmerged=finalmerged.reset_index(drop=True)
finalmerged.to_csv('/Users/Shuxian/Documents/CSYE7390-Spring2016/COB Interval data 2014/finalmerge.csv',index=False)